## Week 1 - Data with Danny

In [ ]:
# | echo: false

from pathlib import Path

from IPython.display import Markdown, display
from sql_8week_danny.sql_engine import DuckDBEngine

In [ ]:
# | echo: false

SQL_DIR = Path.cwd().parent / "sql"
DATA_DIR = Path.cwd().parent / "data"

TABLES_SQL = SQL_DIR / "week1.sql"
DUCK_DB = DATA_DIR / "week1_duckdb.db"

In [ ]:
# | echo: false


def create_database(persist=True):
    if persist and DUCK_DB:
        db = DuckDBEngine(
            db_path=str(DUCK_DB), rm_db=True
        )  # rm the db if it exists to avoid table creation errors
    else:
        db = DuckDBEngine()
    sql = db.execute_sql_file(TABLES_SQL)  # create the tables
    return db, sql

In [ ]:
db, create_sql = create_database(persist=True)

```mermaid
erDiagram
    SALES {
        varchar customer_id "Customer ID"
        date order_date "Order date"
        integer product_id "Product ID"
    }

    MENU {
        integer product_id PK "Unique product ID"
        varchar product_name "Name of the product"
        integer price "Price of the product"
    }

    MEMBERS {
        varchar customer_id PK "Customer ID"
        date join_date "Membership join date"
    }

    SALES ||--|| MENU : "contains"
    SALES }|--|| MEMBERS : "places"
```

In [ ]:
db.table_names

In [ ]:
# | label: display_tables_sql
# | echo: false


def display_sql(sql):
    display(Markdown(f"```\n{sql}"))

In [ ]:
display_sql(create_sql)

In [ ]:
tables_df = db.load_tables_to_df()

#### Table information

In [ ]:
df = db.display_all_table_info(display_tables=True)

In [ ]:
for table in db.table_names:
    display(Markdown(f"# {table}"))
    display(df[table].head())

In [ ]:
db.query("SELECT COUNT(*) FROM members")

In [ ]:
# db.close()  # required to remove the DB file lock (.wal)

In [ ]:
table_info_df = db.query(
    """
    PRAGMA show_tables_expanded; 
    """
)


# PRAGMA table_info('table_name');
# CALL pragma_table_info('table_name'); same info as show_tables_expanded I think
# cid INTEGER,        -- cid of the column
# name VARCHAR,       -- name of the column
# type VARCHAR,       -- type of the column
# notnull BOOLEAN,    -- if the column is marked as NOT NULL
# dflt_value VARCHAR, -- default value of the column, or NULL if not specified
# pk BOOLEAN          -- part of the primary key or not

# SET database_size;  # database size info
# CALL pragma_database_size();

# PRAGMA storage_info('table_name'); # technical table size etc info
# CALL pragma_storage_info('table_name');

# PRAGMA show_databases; get database names

# PRAGMA version; DuckDB version info
# CALL pragma_version();
# PRAGMA user_agent;

In [ ]:
table_info_df

In [ ]:
table_info_df["database"].unique().tolist()[0]

In [ ]:
table_info_df["schema"].unique().tolist()[0]

In [ ]:
table_info_df["name"].unique().tolist()

#### Case Study Questions

Each of the following case study questions can be answered using a single SQL statement:


##### What is the total amount each customer spent at the restaurant?

In [ ]:
db.query(
    "SELECT customer_id, SUM(price) FROM sales as s JOIN menu as m ON s.product_id = m.product_id GROUP BY customer_id"
)

##### How many days has each customer visited the restaurant?

In [ ]:
db.query("SELECT customer_id, COUNT(DISTINCT order_date) FROM sales GROUP BY customer_id")

##### What was the first item from the menu purchased by each customer?

In [ ]:
db.query("SELECT customer_id, MIN(order_date) FROM sales GROUP BY customer_id")

In [ ]:
db.query(
    "SELECT customer_id, product_id FROM sales WHERE order_date = '2021-01-01' GROUP BY customer_id, product_id"
)

##### What is the most purchased item on the menu and how many times was it purchased by all customers?

In [ ]:
db.query(
    "SELECT product_id, COUNT(product_id) FROM sales GROUP BY product_id ORDER BY COUNT(product_id) DESC"
)

##### Which item was the most popular for each customer?

In [ ]:
db.query(
    "SELECT customer_id, product_id, COUNT(product_id) FROM sales GROUP BY customer_id, product_id ORDER BY COUNT(product_id) DESC"
)

##### Which item was purchased first by the customer after they became a member?

In [ ]:
db.query(
    """
    SELECT * FROM sales JOIN members on sales.customer_id = members.customer_id  WHERE sales.customer_id = 'A' AND order_date >= join_date ORDER BY order_date LIMIT 1
    """
)

In [ ]:
db.query(
    """
    SELECT * FROM sales JOIN members on sales.customer_id = members.customer_id  WHERE sales.customer_id = 'B' AND order_date >= join_date ORDER BY order_date LIMIT 1
    """
)

##### Which item was purchased just before the customer became a member?

In [ ]:
db.query(
    """
    SELECT * FROM sales JOIN members on sales.customer_id = members.customer_id  WHERE sales.customer_id = 'A' AND order_date < join_date ORDER BY order_date LIMIT 1
    """
)

In [ ]:
db.query(
    """
    SELECT * FROM sales JOIN members on sales.customer_id = members.customer_id  WHERE sales.customer_id = 'B' AND order_date < join_date ORDER BY order_date LIMIT 1
    """
)

##### What is the total items and amount spent for each member before they became a member?

In [ ]:
db.query(
    """
    SELECT count(*) FROM sales JOIN members on sales.customer_id = members.customer_id  WHERE sales.customer_id = 'A' AND order_date < join_date
    """
)

In [ ]:
# need to join menu table for this

In [ ]:
db.query(
    """
    SELECT SUM(price) FROM sales JOIN members on sales.customer_id = members.customer_id JOIN menu on menu.product_id = sales.product_id  WHERE sales.customer_id = 'A' AND order_date < join_date
    """
)

In [ ]:
db.query(
    """
    SELECT count(*) FROM sales JOIN members on sales.customer_id = members.customer_id  WHERE sales.customer_id = 'B' AND order_date < join_date
    """
)

In [ ]:
db.query(
    """
    SELECT SUM(price) FROM sales JOIN members on sales.customer_id = members.customer_id JOIN menu on menu.product_id = sales.product_id  WHERE sales.customer_id = 'B' AND order_date < join_date
    """
)

##### If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?


##### If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?